In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import rich
import requests
from io import BytesIO

In [2]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI()

## File Search is only part of Responses API

https://platform.openai.com/docs/guides/tools-file-search

File Search works with a vector store, which we will need to create within OpenAI. The entire process, including storing the file, generating embeddings, and searching within the vector store, will be managed by OpenAI

The `create_file` function uploads a file to the OpenAI API and supports both remote and local files.
If the file path is a URL, it downloads the file content, stores it in memory and then upload

In [3]:
# Create function to Upload a file
def create_file(openai, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = openai.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = openai.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    return result.id

##### Call function to upload a file

In [ ]:
file_id = create_file(openai, "https://cdn.openai.com/API/docs/deep_research_blog.pdf")
# file_id = create_file(openai, "docs/Panaversity-Certified-Agentic-and-Robotic-AI-Engineer.pdf")

file-4GoREtVZ14bPdCKHXsva8k


### Create a vector store

In [5]:
vector_store = openai.vector_stores.create(
    name="knowledge_base_2"
)
print(vector_store.id)

vs_67e389ea42008191964bfc8f0d0e76ea


### Add a file to a vector store

In [ ]:
result = openai.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)
rich.print(result)

VectorStoreFile(
    id='file-4GoREtVZ14bPdCKHXsva8k',
    created_at=1742965256,
    last_error=None,
    object='vector_store.file',
    status='in_progress',
    usage_bytes=0,
    vector_store_id='vs_67e389ea42008191964bfc8f0d0e76ea',
    attributes={},
    chunking_strategy=StaticFileChunkingStrategyObject(
        static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800),
        type='static'
    )
)

### Check the status of files in vector store

In [7]:
result = openai.vector_stores.files.list(
    vector_store_id=vector_store.id
)
rich.print(result)

SyncCursorPage[VectorStoreFile](
    data=[
        VectorStoreFile(
            id='file-4GoREtVZ14bPdCKHXsva8k',
            created_at=1742965256,
            last_error=None,
            object='vector_store.file',
            status='completed',
            usage_bytes=66539,
            vector_store_id='vs_67e389ea42008191964bfc8f0d0e76ea',
            attributes={},
            chunking_strategy=StaticFileChunkingStrategyObject(
                static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800),
                type='static'
            )
        )
    ],
    has_more=False,
    object='list',
    first_id='file-4GoREtVZ14bPdCKHXsva8k',
    last_id='file-4GoREtVZ14bPdCKHXsva8k'
)

Now in Responses API call, we can provide `file_search` tool and `vector_store_ids`.

Annotation object in response will provide information about file
```
AnnotationFileCitation(
    file_id='file-4GoREtVZ14bPdCKHXsva8k',
    index=1939,
    type='file_citation',
    filename='deep_research_blog.pdf'
)
```

In [8]:
response = openai.responses.create(
    model=MODEL,
    input="What is deep research by OpenAI?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)
rich.print(response.output[1].content[0].text)
# rich.print(response)
rich.print(response.output)

Deep Research by OpenAI is a new capability that enables users to conduct extensive and complex research tasks 
quickly and efficiently. Some key features include:

1. **Multi-Step Research**: Deep Research allows users to generate outputs through a series of reasoning and data 
synthesis from numerous online sources. It can handle intricate queries that would typically require significant 
human effort.

2. **Independence and Automation**: Users provide a prompt, and the system independently finds, analyzes, and 
synthesizes the required information, generating comprehensive reports similar to what a research analyst might 
produce.

3. **Web Browsing and Data Analysis**: It utilizes a specialized version of OpenAI's model optimized for web 
browsing and data analysis, making it capable of interpreting vast amounts of information, including text, images, 
and PDFs.

4. **Documentation and Citations**: Every result is accompanied by detailed documentation and citations, ensuring 
that users can verify the information and understand the reasoning behind the outputs.

5. **Targeted Use Cases**: Deep Research is particularly beneficial for professionals in finance, science, policy, 
and engineering, as well as consumers seeking personalized recommendations that require extensive research.

6. **Performance**: Initial evaluations show that the model powering Deep Research significantly outperforms 
previous models in expert-level tasks and public evaluations, achieving higher accuracy.

7. **Accessibility and Future Plans**: Currently, Deep Research is available to Pro users, with plans to extend 
access to more user tiers over time, and enhancements will be rolled out to improve the system based on user 
feedback and usage data.

This represents a notable advancement towards OpenAI's broader goal of achieving artificial general intelligence 
(AGI), specifically in the realm of knowledge synthesis and research capabilities.

[
    ResponseFileSearchToolCall(
        id='fs_67e38ae6c04c81928847954bfb6a3d750b92b94f1d963d71',
        queries=['What is deep research by OpenAI?'],
        status='completed',
        type='file_search_call',
        results=None
    ),
    ResponseOutputMessage(
        id='msg_67e38ae9909c8192b29105f1b9a8cadb0b92b94f1d963d71',
        content=[
            ResponseOutputText(
                annotations=[
                    AnnotationFileCitation(
                        file_id='file-4GoREtVZ14bPdCKHXsva8k',
                        index=1495,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-4GoREtVZ14bPdCKHXsva8k',
                        index=1495,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-4GoREtVZ14bPdCKHXsva8k',
                        index=1495,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-4GoREtVZ14bPdCKHXsva8k',
                        index=1495,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-4GoREtVZ14bPdCKHXsva8k',
                        index=1742,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    ),
                    AnnotationFileCitation(
                        file_id='file-4GoREtVZ14bPdCKHXsva8k',
                        index=1939,
                        type='file_citation',
                        filename='deep_research_blog.pdf'
                    )
                ],
                text="Deep Research by OpenAI is a new capability that enables users to conduct extensive and 
complex research tasks quickly and efficiently. Some key features include:\n\n1. **Multi-Step Research**: Deep 
Research allows users to generate outputs through a series of reasoning and data synthesis from numerous online 
sources. It can handle intricate queries that would typically require significant human effort.\n\n2. 
**Independence and Automation**: Users provide a prompt, and the system independently finds, analyzes, and 
synthesizes the required information, generating comprehensive reports similar to what a research analyst might 
produce.\n\n3. **Web Browsing and Data Analysis**: It utilizes a specialized version of OpenAI's model optimized 
for web browsing and data analysis, making it capable of interpreting vast amounts of information, including text, 
images, and PDFs.\n\n4. **Documentation and Citations**: Every result is accompanied by detailed documentation and 
citations, ensuring that users can verify the information and understand the reasoning behind the outputs.\n\n5. 
**Targeted Use Cases**: Deep Research is particularly beneficial for professionals in finance, science, policy, and
engineering, as well as consumers seeking personalized recommendations that require extensive research.\n\n6. 
**Performance**: Initial evaluations show that the model powering Deep Research significantly outperforms previous 
models in expert-level tasks and public evaluations, achieving higher accuracy.\n\n7. **Accessibility and Future 
Plans**: Currently, Deep Research is available to Pro users, with plans to extend access to more user tiers over 
time, and enhancements will be rolled out to improve the system based on user feedback and usage data.\n\nThis 
represents a notable advancement towards OpenAI's broader goal of achieving artificial general intelligence (AGI), 
specifically in the realm 

Customize the number of results you want to retrieve from the vector stores.


You will notice a change in the result only if you also provide `include=["file_search_call.results"]`. This will allow you to see the difference.

In [9]:
response = openai.responses.create(
    model=MODEL,
    input="What is deep research by OpenAI?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        "max_num_results": 2  
    }],
    include=["file_search_call.results"]  # This will include actual portion of text content from the file
)
rich.print(response.output[1].content[0].text)
# rich.print(response)
rich.print(response.output[0].results)
rich.print(response.output[1])

Deep research by OpenAI is a new capability launched in ChatGPT that enables the AI to conduct multi-step research 
on the internet. This feature allows users to offload complex research tasks, synthesizing vast amounts of online 
information into comprehensive reports akin to those of a research analyst. It achieves this through advanced 
reasoning and browsing capabilities optimized for tasks requiring extensive information gathering.

### Key Features
- **Agentic Capability**: Users provide a prompt, and the AI independently finds, analyzes, and synthesizes 
information from hundreds of online sources.
- **Multi-step Tasks**: It can complete complex research tasks that would otherwise take a significant amount of 
human time.
- **Comprehensive Outputs**: Every output is fully documented with citations and summaries, making the information 
easy to reference.
- **Wide Applications**: Particularly useful for intensive knowledge work in sectors like finance, science, policy,
and engineering, as well as for personalized shopping research.

The development of deep research aligns with OpenAI's broader goals towards creating Artificial General 
Intelligence (AGI) capable of generating new scientific knowledge, as it synthesizes information to generate 
valuable insights.

[
    Result(
        attributes={},
        file_id='file-4GoREtVZ14bPdCKHXsva8k',
        filename='deep_research_blog.pdf',
        score=0.9820093016280954,
        text="Introducing deep research | OpenAI\n\n\nFebruary 2, 2025 Release\n\nIntroducing deep research\nAn 
agent that uses reasoning to synthesize large amounts of\nonline information and complete multi-step research 
tasks\nfor you. Available to Pro users today, Plus and Team next.\n\nTry on ChatGPT\n\nListen to article 8:18 
Share\n\n21/02/2025, 19:58 Introducing deep research | 
OpenAI\n\nhttps://openai.com/index/introducing-deep-research/ 
1/38\n\nhttps://openai.com/research/index/release/\nhttps://chatgpt.com/\nhttps://openai.com/\n\n\nToday we’re 
launching deep research in ChatGPT, a new agentic capability that\nconducts multi-step research on the internet for
complex tasks. It accomplishes in\ntens of minutes what would take a human many hours.\n\nDeep research is OpenAI's
next agent that can do work for you independently—you\ngive it a prompt, and ChatGPT will find, analyze, and 
synthesize hundreds of online\nsources to create a comprehensive report at the level of a research analyst. 
Powered\nby a version of the upcoming OpenAI o3 model that’s optimized for web browsing and\ndata analysis, it 
leverages reasoning to search, interpret, and analyze massive amounts\nof text, images, and PDFs on the internet, 
pivoting as needed in reaction to information\nit encounters.\n\nThe ability to synthesize knowledge is a 
prerequisite for creating new knowledge. For\nthis reason, deep research marks a significant step toward our 
broader goal of\ndeveloping AGI, which we have long envisioned as capable of producing novel\nscientific 
research.\n\nWhy we built deep research\n\nDeep research is built for people who do intensive knowledge work in 
areas like\nfinance, science, policy, and engineering and need thorough, precise, and reliable\nresearch. It can be
equally useful for discerning shoppers looking for hyper-\npersonalized recommendations on purchases that typically
require careful research,\nlike cars, appliances, and furniture. Every output is fully documented, with 
clear\ncitations and a summary of its thinking, making it easy to reference and verify the\ninformation. It is 
particularly effective at finding niche, non-intuitive information that\nwould require browsing numerous websites. 
Deep research frees up valuable time by\nallowing you to offload and expedite complex, time-intensive web research 
with just\none query.\n\nDeep research independently discovers, reasons about, and consolidates insights 
from\nacross the web. To accomplish this, it was trained on real-world tasks requiring\nbrowser and Python tool 
use, using the same reinforcement learning methods behind\nOpenAI o1, our first reasoning model. While o1 
demonstrates impressive capabilities in\n\n21/02/2025, 19:58 Introducing deep research | 
OpenAI\n\nhttps://openai.com/index/introducing-deep-research/ 2/38\n\nhttps://openai.com/\n\n\ncoding, math, and 
other technical domains, many real-world challenges demand\nextensive context and information gathering from 
diverse online sources. Deep\nresearch builds on these reasoning capabilities to bridge that gap, allowing it to 
take\non the types of problems people face in work and everyday life.\n\nHow to use deep research\n\nIn ChatGPT, 
select ‘deep research’ in the message composer and enter your query. Tell\nChatGPT what you need—whether it’s a 
competitive analysis on streaming platforms\nor a personalized report on the best commuter bike. You can attach 
files or\nspreadsheets to add context to your question. Once it starts running, a sidebar\nappears with a summary 
of the steps taken and sources used."
    ),
    Result(
        attributes={},
        file_id='file-4GoREtVZ14bPdCKHXsva8k',
        filename='deep_research_blog.pdf',
        score=0.946322956540013,
        text="Deep research\n\nHelp me find iOS and android adopti

ResponseOutputMessage(
    id='msg_67e38bf681bc819290cbe60da20ab788069a0f3d2e156a04',
    content=[
        ResponseOutputText(
            annotations=[
                AnnotationFileCitation(
                    file_id='file-4GoREtVZ14bPdCKHXsva8k',
                    index=1279,
                    type='file_citation',
                    filename='deep_research_blog.pdf'
                )
            ],
            text="Deep research by OpenAI is a new capability launched in ChatGPT that enables the AI to conduct 
multi-step research on the internet. This feature allows users to offload complex research tasks, synthesizing vast
amounts of online information into comprehensive reports akin to those of a research analyst. It achieves this 
through advanced reasoning and browsing capabilities optimized for tasks requiring extensive information 
gathering.\n\n### Key Features\n- **Agentic Capability**: Users provide a prompt, and the AI independently finds, 
analyzes, and synthesizes information from hundreds of online sources.\n- **Multi-step Tasks**: It can complete 
complex research tasks that would otherwise take a significant amount of human time.\n- **Comprehensive Outputs**: 
Every output is fully documented with citations and summaries, making the information easy to reference.\n- **Wide 
Applications**: Particularly useful for intensive knowledge work in sectors like finance, science, policy, and 
engineering, as well as for personalized shopping research.\n\nThe development of deep research aligns with 
OpenAI's broader goals towards creating Artificial General Intelligence (AGI) capable of generating new scientific 
knowledge, as it synthesizes information to generate valuable insights.",
            type='output_text'
        )
    ],
    role='assistant',
    status='completed',
    type='message'
)